# FM Demodulator on PYNQ

## Required Libraries

In this project, we use RTL2832 USB tuners to get our RF input samples. 
<img src="rtl2832.png" alt="RTL2832" style="width: 200px;"/>
In order to interact with this unit, you should install the following library:

https://pypi.org/project/pyrtlsdr/

In [ ]:
! pip3 install scikit-dsp-comm==1.3

### FPGA implementation

Similar to your OFDM design, you can implement *mono_FM* as following:

```
mono_fm(complex input, float output){
    
    lf1(complex input, complex output);  // first linear_filter
    downsample1(complex input, complex output); // first downsample
    discrim(complex input, float output);
    lf2(float input, float output); // second linear_filter
    downsample2(float input, float output); // second downsample
}
```

Possible optimizations:

* Dataflow

* Loop unrolling

* Pipelining

You have to create your own testbecnh and implement moono_FM in Vivado_HLS and then, using Vivado, you have to generate a working bitstream.


## DEMO

In [2]:
from pynq import Overlay
from pynq import allocate
import numpy as np
import time
from IPython.display import Audio
import sk_dsp_comm.rtlsdr_helper as sdr_helper
from IPython.display import Audio

In [3]:
from rtlsdr import *  # importing the library to interact with the RF tuner
import sk_dsp_comm.rtlsdr_helper as sdr

sdr = RtlSdr()  # initializing an instance for the device

# configure device
sdr.sample_rate = 2.4e6  # reading 2.4M samples per second
sdr.center_freq = 94.1e6  # tuning to 94.1MHz
sdr.gain = 40.2

t_s = 3  # we read 3 second worth of samples
n_s = int(sdr.sample_rate)*t_s  # number of samples in 1 second

samples = sdr.read_samples(n_s)  # reading samples to the "samples" array
sdr.close()  # closing the device instance

In [4]:
tic = time.clock()
z_bb, z_out = sdr_helper.mono_FM(samples, fs=2.4e6, file_name='SW.wav')
toc = time.clock()
print("Proceesing time: {} seconds".format(toc - tic))

Audio('SW.wav')

/usr/lib/python3/dist-packages/scipy/signal/signaltools.py:1367: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]


Done!
Proceesing time: 16.089657 seconds


In [5]:
ol = Overlay('fm_burst_4.bit')
ol.download()

In [6]:
mfm_ip = ol.fm_demodulator_0
length = 32000
length50 = int(length/50)
in_r = allocate(shape=(length,), dtype=np.float32)
in_i = allocate(shape=(length,), dtype=np.float32)
out_hw = allocate(shape=(length50,), dtype=np.float32)
z_out_hw = np.zeros((2400*20*t_s,), dtype=np.float32)

tic = time.clock()
for i in range(int(2400000*t_s/length)):
    x = samples[i*length:(i+1)*length]
    x_hw = x.astype(np.complex64)
    
    np.copyto(in_r, np.real(x_hw))
    np.copyto(in_i, np.imag(x_hw))

    mfm_ip.write(0x10, in_r.physical_address)
    mfm_ip.write(0x1c, in_i.physical_address)
    mfm_ip.write(0x28, out_hw.physical_address)
    mfm_ip.write(0x00, 1)
    while not (mfm_ip.read(0x00)&0x2):
        pass
    np.copyto(z_out_hw[i*length50:(i+1)*length50], out_hw)
    
    in_r.flush()
    in_i.flush()
    out_hw.flush()
toc = time.clock()
print("Proceesing time: {} seconds".format(toc - tic))

in_r.close()
in_i.close()
out_hw.close()

Proceesing time: 2.9267910000000015 seconds


In [7]:
Audio(z_out_hw, rate=48000)